## Optional

## Main Code

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [8]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [10]:
# import Libraries
import langchain
import pinecone 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
# from langchain.llms import OpenAI
from langchain_groq import ChatGroq
import os

In [12]:
# Load and read the text file
def read_text_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

file_content = read_text_file('/Users/surya/Downloads/Projects/litzchill_db.txt')

In [13]:
# Split the document into chunks
def chunk_data(text, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc_chunks = text_splitter.split_text(text)
    return doc_chunks

chunks = chunk_data(file_content)

In [14]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client with API key
pc = Pinecone(api_key="pcsk_4xH7Hh_Cz1sssGAqSDpufcpriyDHffPvrxtoeynWMjVsvJ35WD2cmAAEUwfKxAUkXebVfe")

# Create index with 384 dimension if not exists
if 'myindex' not in pc.list_indexes().names():
    pc.create_index(
        name='myindex', 
        dimension=384,
        metric='euclidean',  # or another metric you prefer
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        )
    )


In [15]:
# List existing indexes
print(pc.list_indexes().names())

['myindex', 'litzchill-db', 'aichatbot']


In [16]:
from pinecone import Pinecone

# Directly pass the API key
pc = Pinecone(api_key="pcsk_4xH7Hh_Cz1sssGAqSDpufcpriyDHffPvrxtoeynWMjVsvJ35WD2cmAAEUwfKxAUkXebVfe")

# Example: List all available indexes
print(pc.list_indexes().names())

['myindex', 'litzchill-db', 'aichatbot']


In [17]:
# Use a Hugging Face model for embedding
from langchain_huggingface import HuggingFaceEmbeddings

# Now create the embeddings object
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [18]:
vector = [embeddings.embed_query(chunk) for chunk in chunks]

print(f"Number of chunks: {len(chunks)}")
# print(f"Embedding vector for the first chunk: {vectors[0]}")

Number of chunks: 8


In [19]:
len(vector[0])

384

In [20]:
vector[0]

[-0.04406539350748062,
 -0.08544450253248215,
 -0.0720536857843399,
 -0.011679906398057938,
 -0.04077007248997688,
 -0.011764132417738438,
 0.06060539186000824,
 0.0078201275318861,
 -0.018176555633544922,
 -0.0688597783446312,
 0.028944825753569603,
 0.019084185361862183,
 0.016970055177807808,
 0.06410747766494751,
 0.0015176007291302085,
 -0.012920157983899117,
 0.006781105417758226,
 -0.06283330172300339,
 0.0496576726436615,
 -0.12478870898485184,
 -0.025275181978940964,
 -0.02063898742198944,
 0.024293066933751106,
 -0.06050717830657959,
 0.04915667325258255,
 -0.025969302281737328,
 0.06898877769708633,
 0.008831820450723171,
 -0.02881011553108692,
 -0.0657029077410698,
 0.0622234046459198,
 0.09220544248819351,
 -0.07559008151292801,
 -0.0011195020051673055,
 -0.05743318051099777,
 0.10869207978248596,
 -0.062644362449646,
 0.05549139901995659,
 -0.0015992190456017852,
 -0.05551965534687042,
 0.024054257199168205,
 -0.06454357504844666,
 0.014727804809808731,
 -0.00179705931805

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader

# Load your document (e.g., a text file)
loader = TextLoader("/Users/surya/Downloads/Projects/litzchill_db.txt")  # Provide your document path
docs = loader.load()

# Split the documents into chunks if they are large
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_split = text_splitter.split_documents(docs)

# Convert documents to embeddings and upload them to Pinecone
from langchain.vectorstores import Pinecone

# Initialize Pinecone with the existing index
#index_name = "litzchill-db"  # Your Pinecone index
#vectorstore = Pinecone.from_documents(docs_split, embeddings, index_name=index_name)

In [23]:
import os
from pinecone import Pinecone, ServerlessSpec

# Create an instance of the Pinecone class
pc = Pinecone(api_key = "pcsk_4xH7Hh_Cz1sssGAqSDpufcpriyDHffPvrxtoeynWMjVsvJ35WD2cmAAEUwfKxAUkXebVfe")
index_name = "litzchill-db"
index = pc.Index("litzchill-db")

In [24]:
import os
from pinecone import Pinecone, ServerlessSpec

# Create an instance of the Pinecone class
pc = Pinecone(api_key="pcsk_4xH7Hh_Cz1sssGAqSDpufcpriyDHffPvrxtoeynWMjVsvJ35WD2cmAAEUwfKxAUkXebVfe")

# Create the index object
index_name = "litzchill-db"
index = pc.Index(index_name)

# Get the current length (vector count) of the Pinecone collection
index_stats = index.describe_index_stats()

# The response will have a 'namespaces' key, which contains vector counts for each namespace
# Assuming you are using the default namespace (empty string), you can access the vector count like this:
current_vector_cnt = index_stats['namespaces'].get('default', {}).get('vector_count', 0)

print(f"Current vector count in the index '{index_name}': {current_vector_cnt}")

Current vector count in the index 'litzchill-db': 0


In [25]:
## get the current length of the pine cone collection

#current_vector_cnt  = index.describe_index_stats()['namespaces']['litzchill-db']['vector_count']

In [26]:
upsert_data = [ {
    "id" : "vec"+str(current_vector_cnt + i + 1),
     "values" : vector ,
     "metadata" : {"information" : chunks[i]} } for i , vector in enumerate(vector)]
upsert_data[0]

{'id': 'vec1',
 'values': [-0.04406539350748062,
  -0.08544450253248215,
  -0.0720536857843399,
  -0.011679906398057938,
  -0.04077007248997688,
  -0.011764132417738438,
  0.06060539186000824,
  0.0078201275318861,
  -0.018176555633544922,
  -0.0688597783446312,
  0.028944825753569603,
  0.019084185361862183,
  0.016970055177807808,
  0.06410747766494751,
  0.0015176007291302085,
  -0.012920157983899117,
  0.006781105417758226,
  -0.06283330172300339,
  0.0496576726436615,
  -0.12478870898485184,
  -0.025275181978940964,
  -0.02063898742198944,
  0.024293066933751106,
  -0.06050717830657959,
  0.04915667325258255,
  -0.025969302281737328,
  0.06898877769708633,
  0.008831820450723171,
  -0.02881011553108692,
  -0.0657029077410698,
  0.0622234046459198,
  0.09220544248819351,
  -0.07559008151292801,
  -0.0011195020051673055,
  -0.05743318051099777,
  0.10869207978248596,
  -0.062644362449646,
  0.05549139901995659,
  -0.0015992190456017852,
  -0.05551965534687042,
  0.024054257199168205

In [27]:
index.upsert(vectors= upsert_data , namespace="litz-employee-details")

{'upserted_count': 8}

## Querying the PineCone 

In [28]:
query = "litzchill"
query_vector = embeddings.embed_query(query)
len(query_vector)

384

In [29]:
response = index.query(
    namespace="litz-employee-details",
    vector= query_vector,
    top_k=2,
    # include_values=True,
    include_metadata=True
)

In [30]:
retrived_metadata = [ response["matches"][i]["metadata"]["information"] for i in range( len(response["matches"])) ] 
vector_db_text = " ".join( retrived_metadata)

In [31]:
def query_db(query):
    query_vector = embeddings.embed_query(query)
    response = index.query(
        namespace="litz-employee-details",
        vector= query_vector,
        top_k=2,
        include_metadata=True
    )
    retrived_metadata = [ response["matches"][i]["metadata"]["information"] for i in range( len(response["matches"])) ] 
    vector_db_text = " ".join( retrived_metadata)

    return vector_db_text
    

In [32]:
query_db("what do u know about Groq")

'Litzchill’s Core Focus\n\nLITZHILL is focused on leveraging AI technologies to build innovative applications, with a particular emphasis on advanced machine learning systems and AI-driven recommendation engines. The company has a culture of collaboration and technical excellence, driven by a strong team of experienced professionals and skilled interns. Krishnaveni (Front-end Developer): An intern from VR College, Krishnaveni has played a vital role in the front-end development of Litzchill’s applications. Skilled in Angular, she developed a basic app and provided knowledge transfer sessions (KT) to Hemant, further enhancing his development skills.\n\n\nLitzchill’s Core Focus'

## loading the GROQ llm

In [33]:
GROQ_API_KEY= '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'

In [34]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key using the environment variable name
groq_api_key = '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'

# Print the API key to verify (for testing purposes)
print(groq_api_key)

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


In [35]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Llama-3.1-8b-Instant",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x32d32bc10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x331434890>, model_name='Llama-3.1-8b-Instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [36]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [37]:
# Initialize storage for session-based histories
store = {}

# Function to get or create message history based on session_id
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [38]:
# Define a function to process a query with VectorDB integration
def process_query_with_db(query, session_id="default"):
    # Step 1: Fetch information from VectorDB
    db_info = query_db(query)
    
    # Step 2: Combine DB information with the user's query for the chatbot
    user_query = f"{query}\n\n[Context from DB]: {db_info}"
    
    # Step 3: Update message history and run chatbot flow
    history = get_session_history(session_id)
    response = with_message_history.invoke(
        [HumanMessage(content=user_query)],
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

# Initialize the chatbot with RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [39]:
# Test the chatbot with a query
session_id = "chat_session"
query = "Hi, can you tell me more about hemanth and his work at litzchill"

response = process_query_with_db(query, session_id)
print("Response:", response)

Response: Unfortunately, I couldn't find any information about a person named Hemant working at Litzchill. The context you provided seems to be an internal description of individuals working at Litzchill, specifically Hemant and Krishnaveni.

From the context, it appears that Hemant:

1. Works at Litzchill as an AI/ML intern and front-end developer.
2. Has expanded his role to include front-end development, building a powerful editor UI using Ionic and Angular frameworks.
3. Has also worked on AWS back-end development, utilizing AWS Amplify, GraphQL, and AppSync queries with the assistance of Kiran.
4. Contributed to building AI-based projects at Litzchill, likely with the guidance of Prem and Kiran.

However, without more information, I couldn't find any details about Hemant's specific work at Litzchill or his role within the company. If you have any additional context or details, I'd be happy to try and help further.


In [40]:
# Test the chatbot with a query
session_id = "chat_session"
query = "who is Prem"

response = process_query_with_db(query, session_id)
print("Response:", response)

Response: Prem Shekhar is a Co-founder and Lead AI Engineer at Litzchill. He is responsible for overseeing the company's machine learning initiatives and managing the ML team. 

Some key points about Prem Shekhar:

1. He is an IIT Bombay alumnus, having graduated from the prestigious institution.
2. He brings cutting-edge AI knowledge to the table, which is instrumental in driving Litzchill's machine learning initiatives.
3. He has a close professional relationship with Rahul Kiran Rao, another founder of Litzchill, whom he studied with at IIT Bombay.
4. As a co-founder, Prem is likely deeply involved in the company's strategy and growth, leveraging his AI expertise to drive innovation at Litzchill.
